In [5]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.base import clone

In [3]:
def replaceSeuil(DF , col , seuil) :
    R = DF[col].value_counts()
    Ind = list(R.index)
    R = list(R)
    Replace = Ind[R.index(min(R))]
    for i in range(0 , len(R)):
        if R[i] <= seuil :
            DF.loc[df[col] == Ind[i] , col] = Replace
    return DF
def getChanges(df , Y, column , clf , seuil ):
    DF = df.copy()
    R = DF[column].value_counts()
    Ind = list(R.index)
    R = list(R)
    Replace = Ind[R.index(min(R))]
    for i in range(0 , len(R)):
        if R[i] <= seuil :
            DF.loc[df[column] == Ind[i] , column] = Replace
    return  df 
def getScoreOfSeuil(df , Y, column , clf , seuil ):
    DF = df.copy()
    R = DF[column].value_counts()
    Ind = list(R.index)
    R = list(R)
    Replace = Ind[R.index(min(R))]
    for i in range(0 , len(R)):
        if R[i] <= seuil :
            DF.loc[df[column] == Ind[i] , column] = Replace
    return getScoreClf(DF , Y , clf) , df 
def getBestSeuil(df , Y , clf , score  , List):
    maxS=score
    ins = -1
    DR = df.copy()
    Seuilx = 0
    Choosen = ""
    for col in DR.columns.values :
        if col not in List :
            print col
            for s in np.unique(np.array(DR[col].value_counts()))[:10]:
                sc = getScoreOfSeuil(DR , Y, col , clf , s )
                print("Column :" , col , "Seuil : " , s  , "ROC :" , sc )
                if sc >= maxS :
                    ins = 1
                    maxS = sc 
                    Seuilx = s 
                    Choosen = col
    if ins != -1 :
        print("Choose :" , Choosen , "Roc :" ,  maxS)
        List.append(Choosen)
        Xr = replaceSeuil(df , Choosen , Seuilx)
        return getBestSeuil(Xr , Y , clf , maxS , List)
    else :
        return DR     
def getScoreClf(DF , Y , clf ):
    clf = clone(clf)
    return max(cross_val_score(clf , DF , Y , scoring='roc_auc' ,  cv = 5 , n_jobs=-1))
def getScoreForSeuil(df , Y, column , clf , seuil , score ) :
    DF  = df.copy()
    R = DF[column].value_counts()
    Ind = list(R.index)
    R = list(R)
    Replace = Ind[R.index(min(R))]
    for i in range(0 , len(R)):
        if R[i] <= seuil :
            DF.loc[df[column] == Ind[i] , column] = Replace
    scoreF = getScoreClf(DF , Y , clf)
    if scoreF > score :
        seuil =  seuil + 1
        return getScoreForSeuil(DF , Y , column ,  clf , seuil , scoreF )
    else :
        return seuil , df    
def getBestRelationROC(df , Y , realtions , clf , ScoreM ):
    What = df.copy()
    maxScore = ScoreM
    i = 0
    index = -1
    for relation in realtions:
        print("Doing :" , relation )
        if relation[0] in list(df.columns.values) and relation[1] in list(df.columns.values):
            Score , Df = getScoreRelation( df  , Y ,  relation[0] , relation[1] , clf )
            if Score >= maxScore :
                print("Choose :" , relation )
                maxScore = Score
                What = Df
                index = i
        elif relation[0] in list(df.columns.values) :
            Dec ,Sc ,  DC  =  getDelete(df , Y , relation[0] , clf)
            if Dec == True :
                if Sc >= maxScore :
                    print("Delete :" , relation[0] )
                    maxScore = Sc 
                    What = DC 
                    index = i 
                
        elif relation[1] in list(df.columns.values) :
            Dec ,Sc ,  DC  =  getDelete(df , Y , relation[1] , clf)
            if Dec == True :
                if Sc >= maxScore :
                    print("Delete :" , relation[1] )
                    maxScore = Sc 
                    What = DC 
                    index = i 
        i= i+1
    return index , maxScore , What 
def getScoreDf(DF , Y , clf):
    return max(cross_val_score(clf , DF , Y , scoring='roc_auc' ,  cv = 2 , n_jobs=2))
def getScoreRelation(DF,Y,coli , colj , clf):
    D1 = DF.copy()
    D2 = DF.copy()
    D3 = DF.copy()
    D1 , D2   = D1.drop([coli] , axis = 1) , D2.drop([colj] , axis = 1) 
    Temp = D3[coli].astype(str) + D3[colj].astype(str)
    newName = coli+":"+colj
    D3  = D3.drop([colj] , axis = 1)
    D3 = D3.rename(columns = {coli:newName})
    Scores = [getScoreDf(DF , Y , clf) , getScoreDf(D1 , Y , clf) , getScoreDf(D2 , Y , clf) , getScoreDf(D3 , Y , clf)]
    DataFrames = [DF , D1 , D2 , D3 ]
    return max(Scores) , DataFrames[Scores.index(max(Scores))]

def getRelationOneToN( df , seuil ) :
    #clf = Notre Classifieur 
    #DF = Notre DataFrame 
    # Seuil = Seuil utilise pour  detecter les relations 1->n (meme avec le bruit)
    List = []
    Banned = []
    for i in df.columns.values :
        for j in df.columns.values :
            
            if i != j :
                R = [i , j ]
                P = [j , i ]
                if R not in Banned and P not in Banned :
                    x =  df.groupby(i)[j].apply(lambda f : len(np.unique(f)))
                    if x.quantile(seuil)*1.0 == 1 :
                        List.append(R)
                        Banned.append(R)
                        Banned.append(P)
    return List
def getDelete(DF , Y , ColumnI , clf ):
    DF = DF.copy()
    X1 = DF.drop([ColumnI] , axis =1 )
    clf = clone(clf)
    clf1 = clone(clf)
    sss = StratifiedShuffleSplit(Y, 1, test_size=0.2, random_state=0)
    for train , test in sss:
        Train , Test  = train , test
    clf.fit(DF.ix[Train].values , Y[Train])
    clf1.fit(X1.ix[Train].values , Y[Train])
    
    Score = clf.predict_proba(DF.ix[Test].values)[:,1]
    Score1 = clf1.predict_proba(X1.ix[Test].values)[:,1]
    roc0 = metrics.roc_auc_score(Y[Test] , Score )
    roc1 = metrics.roc_auc_score(Y[Test] , Score1 )
    if roc0 <= roc1 :
        return True ,roc1 , X1
    else :
        return False ,roc0 , DF

In [6]:
df =  pd.read_csv('tr.csv')
df = pd.DataFrame(df)
Y = df.ACTION.values
df =  pd.read_csv('Fin.csv')
df = pd.DataFrame(df)

In [7]:
clf4 = RandomForestClassifier(max_depth = 22 , min_samples_leaf = 9 , min_samples_split = 19 ,n_estimators = 97, random_state=5)

In [18]:
df = replaceSeuil(df , 'ROLE_FAMILY_DESC' , 8)
df = replaceSeuil(df , 'ROLE_CODE' , 4)

In [48]:
Le = [ ]
df = getBestSeuil(df , Y , clf4 , roc1  , Le)

RESOURCE
('Column :', 'RESOURCE', 'Seuil : ', 1, 'ROC :', 0.86563044617268947)
('Column :', 'RESOURCE', 'Seuil : ', 2, 'ROC :', 0.86437272484065886)
('Column :', 'RESOURCE', 'Seuil : ', 3, 'ROC :', 0.86260238484135954)
('Column :', 'RESOURCE', 'Seuil : ', 4, 'ROC :', 0.86663046925014509)
('Column :', 'RESOURCE', 'Seuil : ', 5, 'ROC :', 0.86632367355636042)
('Column :', 'RESOURCE', 'Seuil : ', 6, 'ROC :', 0.8629485466758634)
('Column :', 'RESOURCE', 'Seuil : ', 7, 'ROC :', 0.86586256126145322)
('Column :', 'RESOURCE', 'Seuil : ', 8, 'ROC :', 0.86567334327722134)
('Column :', 'RESOURCE', 'Seuil : ', 9, 'ROC :', 0.86222160664819947)
('Column :', 'RESOURCE', 'Seuil : ', 10, 'ROC :', 0.86153483883816129)
MGR_ID
('Column :', 'MGR_ID', 'Seuil : ', 1, 'ROC :', 0.86760014974704558)
('Column :', 'MGR_ID', 'Seuil : ', 2, 'ROC :', 0.86569732805799793)
('Column :', 'MGR_ID', 'Seuil : ', 3, 'ROC :', 0.86786532680668693)
('Column :', 'MGR_ID', 'Seuil : ', 4, 'ROC :', 0.86889932502715861)
('Column :',

In [8]:
Regle = [['ROLE_CODE' ,  5] , ['ROLE_ROLLUP_2' , 1 ] ,['ROLE_FAMILY' , 4 ] ]

In [58]:
df.to_csv('De.csv',index=False)